In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [2]:
# all imports
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import json
import time

import pandas as pd

from multiprocessing import Pool

import os

In [3]:
airbnb_url = 'https://www.airbnb.es/s/Valencia--Espa%C3%B1a/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=september&flexible_trip_lengths%5B%5D=one_week&date_picker_type=flexible_dates&search_type=autocomplete_click&tab_id=home_tab&query=Valencia&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&source=structured_search_input_header'

In [4]:
soup = BeautifulSoup(requests.get(airbnb_url).content, 'html.parser')

In [5]:
listings = soup.find_all('div', 'cy5jw6o')

In [6]:
# First Generation :)
def extract_basic_features(listing_html):
    features_dict = {}


    name = listing_html.find("div", {"class": "t1jojoys"}).get_text()
    header = listing_html.find("div", {"class": "fb4nyux"}).get_text()
    url = listing_html.find('a').get('href')


    features_dict['name'] = name
    features_dict['header'] = header
    features_dict['url'] = url

    return features_dict

In [7]:
extract_basic_features(listings[0])

{'name': 'Apartamento en València',
 'header': '¡Explora Valencia! wifi, la ciudad de la ciencia y Ruzafa',
 'url': '/rooms/1076976207566141255?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-04-21&check_out=2024-04-26&source_impression_id=p3_1711640210_xLBuMpLnS8hmA0r5&previous_page_section_name=1000&federated_search_id=c203cb0d-ee98-45ec-b633-0931b90cdc54'}

In [8]:
# too many separate extractions
RULES_SEARCH_PAGE = {
    'url': {'tag': 'a', 'get': 'href'},
    'name': {'tag': 'div', 'class': 't1jojoys'},
    'header': {'tag': 'span', 'class': 't6mzqp7'},
    'rating_n_reviews': {'tag': 'span', 'class': 'r1dxllyb'},
    'price': {'tag': 'span', 'class': 'a8jt5op'},
    'superhost': {'tag': 'span', 'class': 'dir dir-ltr'},
}

In [9]:
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]

    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [13]:
print(extract_element(listings[13], RULES_SEARCH_PAGE['header']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['url']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['name']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['price']))
print(extract_element(listings[13], RULES_SEARCH_PAGE['superhost']))

lux Carmen centro histórico
/rooms/19169689?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-10-31&check_out=2024-11-05&source_impression_id=p3_1711640210_lNgyafeNapMavcfd&previous_page_section_name=1000&federated_search_id=c203cb0d-ee98-45ec-b633-0931b90cdc54
Apto. residencial en València
Recomendación del viajero
Anfitrión particular


In [14]:
# for feature in RULES_SEARCH_PAGE:
 #   print(f"{feature}: {extract_element(listings[13], RULES_SEARCH_PAGE[feature])}")

In [15]:
for feature in RULES_SEARCH_PAGE:
    try:
        print(f"{feature}: {extract_element(listings[0], RULES_SEARCH_PAGE[feature])}")
    except:
        print(f"{feature}: empty")

url: /rooms/1076976207566141255?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-04-21&check_out=2024-04-26&source_impression_id=p3_1711640210_xLBuMpLnS8hmA0r5&previous_page_section_name=1000&federated_search_id=c203cb0d-ee98-45ec-b633-0931b90cdc54
name: Apartamento en València
header: ¡Explora Valencia! wifi, la ciudad de la ciencia y Ruzafa
rating_n_reviews: empty
price: Anfitrión profesional
superhost: Anfitrión profesional


In [16]:
airbnb_url

'https://www.airbnb.es/s/Valencia--Espa%C3%B1a/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=september&flexible_trip_lengths%5B%5D=one_week&date_picker_type=flexible_dates&search_type=autocomplete_click&tab_id=home_tab&query=Valencia&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&source=structured_search_input_header'

In [17]:
# let's finally write this function
def get_listings(search_page):
    soup = BeautifulSoup(requests.get(search_page).content, 'html.parser')
    listings = soup.find_all('div', 'cy5jw6o')

    return listings

In [18]:
# it works
len(get_listings(airbnb_url))

18

In [19]:
# let's try next page
new_url = airbnb_url + '&items_offset=20'
len(get_listings(new_url))

18

In [20]:
# checking the content, if the data is there
print(extract_element(get_listings(airbnb_url)[0], RULES_SEARCH_PAGE['name']))
print(extract_element(get_listings(new_url)[0], RULES_SEARCH_PAGE['name']))

Loft en València
Alojamiento en València


In [21]:
# let's iterate through all 15 pages
all_listings = []
for i in range(15):
    offset = 20 * i
    new_url = airbnb_url + f'&items_offset={offset}'
    new_listings = get_listings(new_url)
    all_listings.extend(new_listings)

    # let's check if it's scraping
    print(len(all_listings))

18
36
54
72
90
108
126
144
162
180
198
216
234
252
270


In [22]:
# why? maybe Airbnb tries to prevent scraping
# let's wait a couple of seconds after every iteration
import time

all_listings = []
for i in range(15):
    offset = 20 * i
    new_url = airbnb_url + f'&items_offset={offset}&section_offset=3'
    new_listings = get_listings(new_url)
    all_listings.extend(new_listings)

    # let's check if it's scraping
    print(len(all_listings))

    time.sleep(2)

18
36
54
72
90
108
126
144
162
180
198
216
234
252
270


In [23]:
# another random check, if the data is there
print(extract_element(all_listings[113], RULES_SEARCH_PAGE['name']))

Habitación en València


In [24]:
# 1. build all urls
def build_urls(main_url, listings_per_page=20, pages_per_location=15):
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = main_url + f'&items_offset={offset}'
        url_list.append(url_pagination)

    return url_list

In [25]:
# build a list of URLs
url_list = build_urls(airbnb_url)

In [26]:
url_list

['https://www.airbnb.es/s/Valencia--Espa%C3%B1a/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=september&flexible_trip_lengths%5B%5D=one_week&date_picker_type=flexible_dates&search_type=autocomplete_click&tab_id=home_tab&query=Valencia&monthly_start_date=2024-02-01&monthly_length=3&price_filter_input_type=0&price_filter_num_nights=5&channel=EXPLORE&source=structured_search_input_header&items_offset=0',
 'https://www.airbnb.es/s/Valencia--Espa%C3%B1a/homes?refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=feb

In [27]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'

    return features_dict

In [28]:
extract_page_features(listings[0], RULES_SEARCH_PAGE)

{'url': '/rooms/1076976207566141255?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-04-21&check_out=2024-04-26&source_impression_id=p3_1711640210_xLBuMpLnS8hmA0r5&previous_page_section_name=1000&federated_search_id=c203cb0d-ee98-45ec-b633-0931b90cdc54',
 'name': 'Apartamento en València',
 'header': '¡Explora Valencia! wifi, la ciudad de la ciencia y Ruzafa',
 'rating_n_reviews': 'empty',
 'price': 'Anfitrión profesional',
 'superhost': 'Anfitrión profesional'}

In [29]:
# 2. Iteratively scrape pages
def process_search_pages(url_list):
    features_list = []
    for page in url_list:
        listings = get_listings(page)
        for listing in listings:
            features = extract_page_features(listing, RULES_SEARCH_PAGE)
            features_list.append(features)

    return features_list

In [30]:
# try for one page
base_features = process_search_pages(url_list)

In [31]:
base_features

[{'url': '/rooms/16802543?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-04-01&check_out=2024-04-06&source_impression_id=p3_1711640343_FpTuZL%2Bth5wrSwu0&previous_page_section_name=1000&federated_search_id=807ed5a3-2f6d-4906-a44a-82be1bf2828c',
  'name': 'Loft en València',
  'header': '❤ LOFT4 junto RUZAFA & Ciudad Ciencias_COOL+Jardín',
  'rating_n_reviews': 'empty',
  'price': 'Anfitrión particular',
  'superhost': 'Anfitrión particular'},
 {'url': '/rooms/834737896910608128?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-05-19&check_out=2024-05-24&source_impression_id=p3_1711640343_2JogJUxe4pTjA%2BVi&previous_page_section_name=1000&federated_search_id=807ed5a3-2f6d-4906-a44a-82be1bf2828c',
  'name': 'Apartamento en València',
  'header': 'Apartamento moderno deco deco de 2 CAMAS cerca de CityOfArts & Sciences',
  'rating_n_reviews': 'empty',
  'price': 'Anfitrión particular',
  'superhost': 'Anfitrión particular'},
 {'u

In [32]:
df = pd.DataFrame(base_features)
df

,url,name,header,rating_n_reviews,price,superhost
0,/rooms/16802543?adults=1&children=0&enable_m3_...,Loft en València,❤ LOFT4 junto RUZAFA & Ciudad Ciencias_COOL+Ja...,empty,Anfitrión particular,Anfitrión particular
1,/rooms/834737896910608128?adults=1&children=0&...,Apartamento en València,Apartamento moderno deco deco de 2 CAMAS cerca...,empty,Anfitrión particular,Anfitrión particular
2,/rooms/1079643165443497782?adults=1&category_t...,Habitación en València,"Cerca a la playa, a Museos y al Turía",empty,Recomendación del viajero,Quédate con Elisabeth
3,/rooms/11018408?adults=1&category_tag=Tag%3A86...,Habitación en València,"Si eres digital nomads, tú HB está AQUÍ con fi...",empty,Recomendación del viajero,Quédate con Sagrario
4,/rooms/665147005344309398?adults=1&category_ta...,Habitación en València,Amplia y céntrica habitación en Valencia,empty,Quédate con Natacha,Quédate con Natacha
...,...,...,...,...,...,...
265,/rooms/50849410?adults=1&children=0&enable_m3_...,Loft en València,Loft casi nuevo,empty,Anfitrión particular,Anfitrión particular
266,/rooms/35340024?adults=1&children=0&enable_m3_...,Apartamento en Valencia,El Apartamento Verde Valencia - Ciudad de Las ...,empty,Recomendación del viajero,Anfitrión particular
267,/rooms/1066609139205088031?adults=1&children=0...,Apartamento en València,HOB Las Ciencias Apartments III,empty,Superanfitrión,Anfitrión profesional
268,/rooms/50410838?adults=1&children=0&enable_m3_...,Loft en València,Loft entero en Ruzafa,empty,Superanfitrión,Anfitrión particular


In [33]:
# df['ciudad'] = df['name'].str.replace('Villa en |Cueva en |Vivienda en |Apartamento en |Habitación en |Loft en |Minicasa en |Adosado en  ', '', regex=True)
df['ciudad'] = "Valencia"
df

,url,name,header,rating_n_reviews,price,superhost,ciudad
0,/rooms/16802543?adults=1&children=0&enable_m3_...,Loft en València,❤ LOFT4 junto RUZAFA & Ciudad Ciencias_COOL+Ja...,empty,Anfitrión particular,Anfitrión particular,Valencia
1,/rooms/834737896910608128?adults=1&children=0&...,Apartamento en València,Apartamento moderno deco deco de 2 CAMAS cerca...,empty,Anfitrión particular,Anfitrión particular,Valencia
2,/rooms/1079643165443497782?adults=1&category_t...,Habitación en València,"Cerca a la playa, a Museos y al Turía",empty,Recomendación del viajero,Quédate con Elisabeth,Valencia
3,/rooms/11018408?adults=1&category_tag=Tag%3A86...,Habitación en València,"Si eres digital nomads, tú HB está AQUÍ con fi...",empty,Recomendación del viajero,Quédate con Sagrario,Valencia
4,/rooms/665147005344309398?adults=1&category_ta...,Habitación en València,Amplia y céntrica habitación en Valencia,empty,Quédate con Natacha,Quédate con Natacha,Valencia
...,...,...,...,...,...,...,...
265,/rooms/50849410?adults=1&children=0&enable_m3_...,Loft en València,Loft casi nuevo,empty,Anfitrión particular,Anfitrión particular,Valencia
266,/rooms/35340024?adults=1&children=0&enable_m3_...,Apartamento en Valencia,El Apartamento Verde Valencia - Ciudad de Las ...,empty,Recomendación del viajero,Anfitrión particular,Valencia
267,/rooms/1066609139205088031?adults=1&children=0...,Apartamento en València,HOB Las Ciencias Apartments III,empty,Superanfitrión,Anfitrión profesional,Valencia
268,/rooms/50410838?adults=1&children=0&enable_m3_...,Loft en València,Loft entero en Ruzafa,empty,Superanfitrión,Anfitrión particular,Valencia


In [34]:
ruta = 'Alojamientos_Valencia_Airbnb.csv'
df.to_csv(ruta,index=False)